<a href="https://colab.research.google.com/github/andrey56-ruban/Iowa_liquor_sales/blob/main/Ruban_iowa_liquor_sales_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using BigQuery's Client Library
There are multiple means to work with BigQuery while using Colaboratory, including through [magics](https://colab.sandbox.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/01.03-Magic-Commands.ipynb) and `pandas-gbq`. This example will work through the BigQuery Python client (`google-cloud-bigquery`). If you want more information on working through magics, see the [library reference documentation](https://googleapis.github.io/google-cloud-python/latest/bigquery/magics.html). For an example, see the [Getting started with BigQuery](https://colab.research.google.com/notebooks/bigquery.ipynb) notebook.

See the following resources for more information about using the `google-cloud-bigquery` library:

*   [API methods table](https://googleapis.github.io/google-cloud-python/latest/bigquery/generated/google.cloud.bigquery.client.Client.html#google.cloud.bigquery.client.Client)
*  [BigQuery documentation](https://cloud.google.com/bigquery/docs)
*  [Library reference documentation](https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html)

## Install `plotnine`

This notebook uses the `plotnine` library which is not included with Colaboratory by default. `plotnine` is a visualization library similar to the  `ggplot2` library widely popular among R users thanks to its ability to quickly and consistently develop useful data visualizations.

In [ ]:
!pip install 'plotnine[all]'

     |████████████████████████████████| 8.7MB 13.2MB/s 


In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


### Declare your project
Insert your project ID in between the quotation marks in the cell below. This sets the GCP project used throughout the notebook. You can find this information under the "Project Info" card on the [GCP Console Dashboard](https://console.cloud.google.com).

If you decide to change projects, make sure you come back to this step to replace the project ID and then rerun the steps below.

In [2]:
project_id = "static-destiny-94216"

### Declare your dataset

You also need to name a [BigQuery dataset](https://cloud.google.com/bigquery/docs/datasets-intro) to save some work in. The default is `samples_demo`, but you can change it.

It is recommended you use a dataset just for this example. Code to create the dataset specified here is after the Setup section. You may run into a problems using a dateset that already has other data in it.

In [3]:
dataset_name = "retail_demo"

### Import libraries and instantiate the BigQuery client

Next, import the required libraries, including the `google-cloud-bigquery` library. For convenience, Colaboratory already has the libraries necessary for calling Google Cloud APIs already installed. The other imported libraries aid the analysis.

After importing libraries, the BigQuery "client" is created, which tells the `google-cloud-bigquery` library to connect to your project.

In [4]:
from google.cloud import bigquery
import numpy as np
import pandas as pd
import plotnine as pn
import threading
import time

client = bigquery.Client(project = project_id)

## Create your dataset

If the dataset you specified in Setup does not exist, create it.

You'll see an error when you run this code if the dataset already exists. As stated above, it is recommended you use a dataset just for this example. But if your intended dataset already exists and you are comfortable using it, there is no harm in proceeding past this error.

In [5]:
dataset_id = "{}.{}".format(client.project, dataset_name)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)

# Send the dataset to the API for creation.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
dataset = client.create_dataset(dataset)  # API request
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Conflict: ignored

Изучим данные в первых 3 строках датасета bigquery-public-data.iowa_liquor_sales.sales

In [6]:
head_query = """
    SELECT
     *
    FROM 
      `bigquery-public-data.iowa_liquor_sales.sales`
      LIMIT 3
    """
head_query = client.query(head_query).to_dataframe()
head_query

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,S03701600037,2012-01-24,3477,Sam's Club 6472 / Council Bluffs,3221 MANAWA CENTRE DR,COUNCIL BLUFFS,51501,POINT (-95.847163 41.225208),78,Pottawattamie,1081700,DISTILLED SPIRITS SPECIALTY,434,Luxco-St Louis,75087,Juarez Gold Dss,12,1000,4.75,7.13,96,684.48,96.0,25.36
1,S06230800056,2012-06-25,3932,Main Street Spirits / Mapleton,311 MAIN ST,MAPLETON,51034,POINT (-95.79375 42.165915),67,Monona,1081010,AMERICAN AMARETTO,434,Luxco-St Louis,76487,Paramount Amaretto,12,1000,4.04,6.06,5,30.30,5.0,1.32
2,INV-18932300070,2019-04-23,2643,Hy-Vee Wine and Spirits / Waterloo,2126 Kimball Ave,Waterloo,50701,None,7,BLACK HAWK,1081500,Triple Sec,434,LUXCO INC,86251,Juarez Triple Sec,12,1000,2.42,3.63,2,7.26,2.0,0.52


В датасете есть 24 столбца. Для построения аналитики в данном случае будем ипользовать только часть. Стоит отметить, что есть вычисляемые столбцы, которые можно не загружать если не хватает системных ресурсов: Sale (Dollars), Volume Sold (Liters), Volume Sold (Gallons)

Описание всех столбцов датасета можно прочитать по ссылке: https://data.iowa.gov/Sales-Distribution/Iowa-Liquor-Sales/m3tr-qhgy


Оценим объем датасета в количестве строк, а так же узнаем период, за который доступна история продаж и присутствуют ли продажи за все месяцы

In [16]:
stat_query = """
    SELECT
     count(*) as rows_count, 
     min(date) as min_date_orders,
     max(date) as max_date_orders,
     count(distinct DATE(EXTRACT(YEAR FROM date),EXTRACT(MONTH FROM date),1)) as count_month
    FROM 
      `bigquery-public-data.iowa_liquor_sales.sales`
    """
stat_query = client.query(stat_query).to_dataframe()
stat_query

,rows_count,min_date_orders,max_date_orders,count_month
0,19884006,2012-01-03,2020-10-30,106


Объем датасета более 19 млн строк. Продажи предсталены за каждый месяц начина с 2012 по октябрь 2020. Получим часть данных из источника по наиболее информативным столбцам и за период с начала 2018 года, так как для выводов о текущей сложившейся ситуации на рынке алкогольных напитков Айовы и установления причинно-следственных связей достаточно данных за несколько последних лет

In [17]:
df = """
    SELECT
     date, store_number, city, county, category_name, vendor_name, 
     item_description, bottle_volume_ml/1000 as bottle_volume_liters, state_bottle_cost, state_bottle_retail,
     bottles_sold, sale_dollars, volume_sold_liters
    FROM 
      `bigquery-public-data.iowa_liquor_sales.sales`
      where date>='2018-01-01'
    """
df = client.query(df).to_dataframe()
df

,date,store_number,city,county,category_name,vendor_name,item_description,bottle_volume_liters,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters
0,2018-10-18,3420,Windsor Heights,Polk,Imported Distilled Spirit Specialty,SAZERAC COMPANY INC,Tortilla Gold Dss,1.000,4.85,7.28,768,5199.36,768.00
1,2019-12-05,2675,Coralville,JOHNSON,Aged Dark Rum,DIAGEO AMERICAS,Ron Zacapa 23YR,0.750,22.50,33.75,18,607.50,13.50
2,2018-03-08,3420,Windsor Heights,Polk,None,SAZERAC COMPANY INC,Members Mark Silver Tequila,1.750,13.96,20.94,288,6030.72,504.00
3,2018-10-24,4201,Sioux City,WOODBURY,Gold Rum,LUXCO INC,Paramount Gold Rum,1.750,7.84,11.76,3,35.28,5.25
4,2019-12-19,2548,Altoona,POLK,Straight Rye Whiskies,Infinium Spirits,Templeton 4YR Rye,0.750,18.09,27.14,18,488.52,13.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6884731,2018-02-27,3385,Cedar Rapids,LINN,American Dry Gins,PERNOD RICARD USA,Seagrams Extra Dry Gin,1.000,7.49,11.24,24,269.76,24.00
6884732,2018-09-04,2647,Cedar Rapids,LINN,American Dry Gins,PERNOD RICARD USA,Seagrams Extra Dry Gin,1.000,7.49,11.24,1,11.24,1.00
6884733,2018-04-23,2573,Muscatine,MUSCATINE,Straight Bourbon Whiskies,DIAGEO AMERICAS,Bulleit Bourbon,0.375,7.49,11.24,3,33.72,1.12
6884734,2018-11-20,2590,Cedar Rapids,LINN,Straight Bourbon Whiskies,DIAGEO AMERICAS,Bulleit Bourbon,0.375,7.49,11.24,2,22.48,0.75


Проверим в каких столбцах присутствуют пропущенные значения и в каком количестве

In [18]:
print('Missing values for every column')
print(df.isnull().sum())

Missing values for every column
date                       0
store_number               0
city                    6592
county                  6592
category_name           5621
vendor_name                3
item_description           0
bottle_volume_liters       0
state_bottle_cost          0
state_bottle_retail        0
bottles_sold               0
sale_dollars               0
volume_sold_liters         0
dtype: int64


Так как пропущенные значения присутствуют в текстовых строках, то удалять их не будем из-за влияния на общие числовые показатели. Заменим пропущенные значения на значение No_name во всех столбцах

In [19]:
df.fillna('No_name', inplace=True)

Узнаем какие типы данных у каждого столбца, так как в дальнейшем нужно эффективно использовать ограниченую оперативную память и производить вычисления с использованием данных числовых столбцов

In [20]:
df.dtypes

date                     object
store_number             object
city                     object
county                   object
category_name            object
vendor_name              object
item_description         object
bottle_volume_liters    float64
state_bottle_cost       float64
state_bottle_retail     float64
bottles_sold              int64
sale_dollars            float64
volume_sold_liters      float64
dtype: object

Часть столбцов имеет общий тип - object, необходимо задать каждому столбцу его тип

Столбец с датами переведем в тип дата-время, остальным задаем текстовый, целочисленный или дробный тип, в зависимости от данных в них содержащихся, которые видели на предпросмотре выше

In [21]:
df['date'] = pd.to_datetime(df['date'])

In [22]:
df.astype({'store_number': 'int32',
           'city': 'string',
           'county': 'string',
           'category_name': 'string',
           'vendor_name': 'string',
           'item_description': 'string'
           }).dtypes

date                    datetime64[ns]
store_number                     int32
city                            string
county                          string
category_name                   string
vendor_name                     string
item_description                string
bottle_volume_liters           float64
state_bottle_cost              float64
state_bottle_retail            float64
bottles_sold                     int64
sale_dollars                   float64
volume_sold_liters             float64
dtype: object

Меня заинтересовали данные о количестве населения в каждом городе штата Айова, чтобы применить их для дополнительного анализа. Данные взяты с открытого источника. Импортирована таблица с сайта и представлена в виде датафрейма.

In [23]:
city_population=pd.read_html('https://worldpopulationreview.com/states/cities/iowa', index_col=0)[0]
city_population = city_population.drop(['2010 Census','Change','Density (km²)'], axis=1)

city_population.astype({'Name': 'string',
                       '2020 Pop': 'float64'
                       }).dtypes
           
city_population

,Name,2020 Pop
Rank,,
1,Des Moines,217891
2,Cedar Rapids,135502
3,Davenport,101965
4,Sioux City,81382
5,Iowa City,78440
...,...,...
496,Bussey,407
497,Earling,405
498,Cumming,405


Добавим к существующему основному набору данных данные о популяции через левосторонее объединение

In [24]:
df = pd.merge(df, city_population, left_on=['city'], right_on=['Name'], how="left")

Проверим сколько пропущенных значений с величиной населения города

In [25]:
print('Part of missing values for every column')
print(df.isnull().sum())

Part of missing values for every column
date                        0
store_number                0
city                        0
county                      0
category_name               0
vendor_name                 0
item_description            0
bottle_volume_liters        0
state_bottle_cost           0
state_bottle_retail         0
bottles_sold                0
sale_dollars                0
volume_sold_liters          0
Name                    88867
2020 Pop                88867
dtype: int64


Посмотрим каким городам не нашлось соответсвие в исходном датасете

In [27]:
df['city'][df.isnull().any(axis=1)].unique()

array(['Leclaire', 'St Lucas', 'Delaware', 'ClearLake', 'No_name',
       "Arnold's Park", 'Fort Atkinson', 'Tipton', 'Floyd', 'Mt Pleasant',
       'Zwingle', 'Lemars', 'Jewell', 'Otumwa', 'Holy Cross', 'Amana',
       'Wesley', 'St Ansgar', 'Baldwin', 'MUSCATINE', 'Bevington',
       'Corwith', 'Lost Nation', 'Martelle', 'Harpers Ferry', 'Livermore',
       'Arlington', 'Nichols', 'SAGEVILLE', 'LoneTree', 'St Charles',
       'Templeton', 'Grand Mounds', 'Rudd', 'Saint Ansgar',
       'Pleasant Valley', 'Middletown', 'Lohrville', 'Casey', 'Colesburg',
       'Washta', 'Colorado Springs'], dtype=object)

Просмотрел каждый город, сравнивая исходный и полученный датасет из веба, у части городов несовпали имена - выполним замену имени, у остальных количество населения менее 500 человек, потому население можно не заполнять так как их небольшое количество и учитывая размер населения - несущественное влияние на продажи алкоголя в штате 

In [28]:
df.replace({'city' : {'St Ansgar' : 'St. Ansgar',
                      'Saint Ansgar' : 'St. Ansgar',
                      'Lemars' : 'Le Mars',
                      'Grand Mounds' : 'Grand Mound',
                      'MUSCATINE' : 'Muscatine',
                      'ClearLake' : 'Clear Lake',
                      'Mt Pleasant' : 'Mount Pleasant',
                      'Fort Atkinson' : 'Atkins',
                      "Arnold's Park" : 'Arnolds Park',
                      'Jewell' : 'Jewell Junction',
                      'LoneTree' : 'Lone Tree',
                      'St Charles' : 'St. Charles'
                      }}, inplace=True)

Удалим столбцы ранее присоединенные. Если ресурсы ограничены, то естественно данный фрейм можно не присоединять к исходному, а все сравнения и объединения выполнять непосредстивенно перед построением аналитики на небольших наборах

In [29]:
df = df.drop(["Name", "2020 Pop"], axis=1)

Переименуем столбцы

In [30]:
city_population.rename(columns={"Name": "city", "2020 Pop": "city_population"}, inplace=True)
city_population.reset_index()

,Rank,city,city_population
0,1,Des Moines,217891
1,2,Cedar Rapids,135502
2,3,Davenport,101965
3,4,Sioux City,81382
4,5,Iowa City,78440
...,...,...,...
495,496,Bussey,407
496,497,Earling,405
497,498,Cumming,405
498,499,New Vienna,404


In [31]:
df = pd.merge(df, city_population, on=['city'], how="left")
df

,date,store_number,city,county,category_name,vendor_name,item_description,bottle_volume_liters,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,city_population
0,2018-10-18,3420,Windsor Heights,Polk,Imported Distilled Spirit Specialty,SAZERAC COMPANY INC,Tortilla Gold Dss,1.000,4.85,7.28,768,5199.36,768.00,4774.0
1,2019-12-05,2675,Coralville,JOHNSON,Aged Dark Rum,DIAGEO AMERICAS,Ron Zacapa 23YR,0.750,22.50,33.75,18,607.50,13.50,23106.0
2,2018-03-08,3420,Windsor Heights,Polk,No_name,SAZERAC COMPANY INC,Members Mark Silver Tequila,1.750,13.96,20.94,288,6030.72,504.00,4774.0
3,2018-10-24,4201,Sioux City,WOODBURY,Gold Rum,LUXCO INC,Paramount Gold Rum,1.750,7.84,11.76,3,35.28,5.25,81382.0
4,2019-12-19,2548,Altoona,POLK,Straight Rye Whiskies,Infinium Spirits,Templeton 4YR Rye,0.750,18.09,27.14,18,488.52,13.50,20048.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6884731,2018-02-27,3385,Cedar Rapids,LINN,American Dry Gins,PERNOD RICARD USA,Seagrams Extra Dry Gin,1.000,7.49,11.24,24,269.76,24.00,135502.0
6884732,2018-09-04,2647,Cedar Rapids,LINN,American Dry Gins,PERNOD RICARD USA,Seagrams Extra Dry Gin,1.000,7.49,11.24,1,11.24,1.00,135502.0
6884733,2018-04-23,2573,Muscatine,MUSCATINE,Straight Bourbon Whiskies,DIAGEO AMERICAS,Bulleit Bourbon,0.375,7.49,11.24,3,33.72,1.12,23627.0
6884734,2018-11-20,2590,Cedar Rapids,LINN,Straight Bourbon Whiskies,DIAGEO AMERICAS,Bulleit Bourbon,0.375,7.49,11.24,2,22.48,0.75,135502.0


In [32]:
df['city'][df.isnull().any(axis=1)].unique()

array(['Leclaire', 'St Lucas', 'Delaware', 'No_name', 'Tipton', 'Floyd',
       'Zwingle', 'Otumwa', 'Holy Cross', 'Amana', 'Wesley', 'Baldwin',
       'Bevington', 'Corwith', 'Lost Nation', 'Martelle', 'Harpers Ferry',
       'Livermore', 'Arlington', 'Nichols', 'SAGEVILLE', 'Templeton',
       'Rudd', 'Pleasant Valley', 'Middletown', 'Lohrville', 'Casey',
       'Colesburg', 'Washta', 'Colorado Springs'], dtype=object)

Заменим количество населения в мелких населенных пунктах на 200, чтобы не было пустых значений в датасете

In [33]:
df['city_population'].fillna(200, inplace=True) 
print('Missig values for every column')
print(df.isnull().sum())

Missig values for every column
date                    0
store_number            0
city                    0
county                  0
category_name           0
vendor_name             0
item_description        0
bottle_volume_liters    0
state_bottle_cost       0
state_bottle_retail     0
bottles_sold            0
sale_dollars            0
volume_sold_liters      0
city_population         0
dtype: int64


In [34]:
!pip install chart_studio


     |████████████████████████████████| 71kB 6.1MB/s 


Вызовем необходимые библиотеки для отображения графиков.
Для plotly.io, где нужно указать среду выполнения colab

In [35]:
from chart_studio.plotly import iplot
from plotly import graph_objs as go
from plotly.subplots import make_subplots
#import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'colab'

In [36]:
import warnings
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="white", palette="muted", color_codes=True)

Вычислим среднее потребление алкоголя в литрах на 1 жителя в среднем за месяц в 2020 году по населенным пунктам свыше 2000 жителей и отобразим топ-10 городов по наибольшему потреблению

In [37]:
population_cons = df[df.date.dt.year==2020][['city','city_population', 'volume_sold_liters']].groupby(['city','city_population']).agg({'volume_sold_liters' : 'sum'})
population_cons.columns = ['sum_volume_sold_liters']
population_cons=population_cons.sort_values(['sum_volume_sold_liters'], ascending=False).reset_index()
population_cons['avg_litres_person']=(population_cons['sum_volume_sold_liters']/population_cons['city_population'])/10
population_cons = population_cons[population_cons['city_population']>=5000]
population_cons=population_cons.sort_values(['avg_litres_person'], ascending=False)
avg_litres_person=population_cons[['city','avg_litres_person']].head(10)
population_cons=population_cons.sort_values(['city_population'], ascending=False)
top_city=population_cons[['city', 'city_population', 'avg_litres_person']].head(10)

fig = make_subplots(rows=1, cols=1, shared_xaxes=True,
                    shared_yaxes=False, vertical_spacing=0.001)

trace_b=go.Bar(
    x=avg_litres_person.avg_litres_person,
    y=avg_litres_person.city,
    marker=dict(
                color='rgba(50, 171, 96, 0.6)',
                line=dict(
                color='rgba(50, 171, 96, 1.0)',
                width=1),
               ),
              name='avg_litres_person',
              orientation='h',
)
layout = go.Layout(autosize=True, title= 'Average sales by person in liters', showlegend=True, width=1200, height=470)
fig = go.Figure(data=trace_b, layout=layout)
fig.show()

Наибольшее потребление в городе Spirit Lake - 3 литра на одного человека

In [40]:
fig = make_subplots(rows=1, cols=1,
                    specs=[[{"secondary_y": True}]])

# Top left
trace_b=go.Bar(
    x=top_city.city,
    y=top_city.city_population,
    marker=dict(
                color='rgba(50, 171, 96, 0.6)',
                line=dict(
                color='rgba(50, 171, 96, 1.0)',
                width=1),
               ),
              name='population',
              orientation='v',
              yaxis='y1',
              text=round(top_city.city_population, 1), textposition='outside'
               
)

trace_a = go.Scatter(x=top_city.city, y= top_city.avg_litres_person, yaxis='y2', mode= 'lines+markers', name= 'litres/person', line=dict(width=3))

layout = go.Layout(autosize=True, title= 'Average sales by person in liters in TOP-10 city by population',
                   yaxis2=dict(title='Moddel Difference',overlaying='y', side='right'), showlegend=True, width=1100, height=470)
fig = go.Figure(data=[trace_b, trace_a], layout=layout)
fig.show()

Как видно из двух графиков выше, в отличии от городов с невысокой численностью, в самых крупных городах штата среднее потребление алкоголя на 1 человека в месяц не превышает 1,2 литра. Данная информация полезна для оценки недопродаж по населенным пунктам, что повлечет за собой установление причин низких продаж и действий по их стимулированию

Построим гистограмму суммарных доходов торговых точек от продаж алкоголя за первые 10 месяцев каждого года

In [41]:
year_10m_litres = df[df.date.dt.month<=10].groupby(df.date.dt.year)['volume_sold_liters'].sum().to_frame()

trace_l = go.Bar(
    y= year_10m_litres.volume_sold_liters, x= ['2018','2019','2020'],
    marker=dict(color='rgba(0,0,255,0.4)', line=dict(color='blue', width=1)),
    xaxis='x2',
    yaxis='y2',
    name='Total sales by 10 month year in liters', orientation='v', text=year_10m_litres.volume_sold_liters, textposition='outside', texttemplate='%{text:.2s}'
)

year_10m_sum = df[df.date.dt.month<=10].groupby(df.date.dt.year)['sale_dollars'].sum().to_frame()

trace_s = go.Bar(
    y= year_10m_sum.sale_dollars, x= ['2018','2019','2020'],
    marker=dict(color='rgba(0,255,0,0.4)', line=dict(color='blue', width=1)),
    name='Total sales by 10 month year in USD', orientation='v', text=year_10m_sum.sale_dollars, textposition='outside', texttemplate='%{text:.2s}'
)
layout = go.Layout(autosize=True, title= 'Total sales by 10 month year in USD and liters', showlegend=True, width=1200, height=470,
                   xaxis = dict(domain = [0.0, 0.45]),
                   xaxis2 = dict(domain = [0.55, 1.0]),
                   yaxis2 = dict(overlaying='y',
                                 anchor = 'free',
                                 position = 0.55
                                )
                   )

fig = go.Figure(data=[trace_s, trace_l], layout=layout)


fig.update_layout(showlegend=True,
                  annotations=[
        dict(
            x=2019, y=260000000,
            text="+3,7%",
            showarrow=False,
            yshift=10
            ),
          dict(
            x=2020, y=300000000,
            text="+14,2%",
            showarrow=False,
            yshift=10
            ),
          dict(
            xref='x2', 
            yref='y2',
            x=2019, y=17000000,
            text="+2,8%",
            showarrow=False,
            yshift=10
            ),
          dict(
            xref='x2', 
            yref='y2',
            x=2020, y=18500000,
            text="+8,1%",
            showarrow=False,
            yshift=10
            )
                  ]
)

fig.show()

В 2020 году за первые 10 месяцев продажи выросли на 14,2% в USD и на 8,1% в литрах относительно аналогичного периода 2019, что на 10,5 пп и 6,1 пп выше чем 2019 к 2018. То есть рынок алкоголя в 2020 показывает существенный прирост. По цене прирост выше чем в литрах, посмотрим как изменилась средняя цена за литр.

In [42]:
year_10m_litres['avg_price_litres'] = year_10m_sum['sale_dollars'] / year_10m_litres['volume_sold_liters']
trace_с = go.Bar(
    y= year_10m_litres.avg_price_litres, x= ['2018','2019','2020'],
    marker=dict(color='rgba(0,0,255,0.4)', line=dict(color='blue', width=1)),
    name='Average cost per litres', orientation='v', text=round(year_10m_litres.avg_price_litres, 1), textposition='outside'
)
layout = go.Layout(autosize=True, title= 'Average cost per litres', showlegend=True, width=600, height=470)
fig = go.Figure(data=[trace_с], layout=layout)
fig.show()

Цена за литр в 2020 выросла на 80 центов относительно 2019 года. Было или незначительное повышение цен, или предпочтения потребителей сместились в сторону более дорогих сегментов продукции

Следует посмотреть помесячную динамику потребления алкоголя в литрах

In [43]:
monthly_df = df.groupby([df.date.dt.year, df.date.dt.month])['volume_sold_liters'].sum()
monthly_df.index = monthly_df.index.set_names(['year', 'month'])
monthly_df = monthly_df.reset_index()
traces = []
for i in range(2018, 2021):
    store_sales = monthly_df[monthly_df.year == i]
    trace = go.Scatter(x= monthly_df.month, y= store_sales.volume_sold_liters, mode= 'lines+markers', name= str(i), line=dict(width=3))
    traces.append(trace)
layout = go.Layout(autosize=True, title= 'Sales in liters per month', showlegend=True)
fig = go.Figure(traces, layout=layout)
fig.show()

Динамика показывает, что значительный скачки продаж, нетипичный для двух предыдущих годов произошли в марте 2020. Согласно https://en.wikipedia.org/wiki/COVID-19_pandemic_in_Iowa в марте это могло быть из-за паники, вызванной Covid. Таким образом произошел сдвиг пиков сезонного спроса из-за накопленных запасов с мая на июнь.

In [44]:
monthly_df = df.groupby([df.date.dt.year, df.date.dt.month])['volume_sold_liters'].sum()
monthly_df.index = monthly_df.index.set_names(['year', 'month'])
monthly_df = monthly_df.reset_index()
x_axis = []
for y in range(2018, 2021):
    for m in range(1,13):
        x_axis.append("{}/{}".format(m,y))
trace = go.Scatter(x= x_axis, y= monthly_df.volume_sold_liters, mode= 'lines+markers', name= 'sales avg per month', line=dict(width=3))
layout = go.Layout(autosize=True, title= 'Sales in liters per month', showlegend=True)
fig = go.Figure(data=[trace], layout=layout)
fig.show()

Посмотрим как изменились покупательские предпочтения в течении трех лет по категориям продукции.

In [45]:
category_percent= df.groupby([df.date.dt.year, 'category_name']).agg({'volume_sold_liters': 'sum'})
category_percent= category_percent.groupby(level=0).apply(lambda x: 100*x / float(x.sum()))
category_percent=category_percent.sort_values(['date','volume_sold_liters'], ascending=False).reset_index()
category_percent=category_percent.groupby(['date']).head(5)
fig = go.Figure(data=[
                      go.Bar(name='2020', x=category_percent[category_percent['date']==2020].category_name, 
                      y=category_percent[category_percent['date']==2020].volume_sold_liters, 
                      text=round(category_percent[category_percent['date']==2020].volume_sold_liters, 0), textposition='outside'),
                      go.Bar(name='2019', x=category_percent[category_percent['date']==2019].category_name, 
                      y=category_percent[category_percent['date']==2019].volume_sold_liters, 
                      text=round(category_percent[category_percent['date']==2019].volume_sold_liters, 0), textposition='outside'), 
                      go.Bar(name='2018', x=category_percent[category_percent['date']==2018].category_name, 
                      y=category_percent[category_percent['date']==2018].volume_sold_liters, 
                      text=round(category_percent[category_percent['date']==2018].volume_sold_liters, 0), textposition='outside')            
])   
fig.update_layout(barmode='group', title= 'TOP-5 sales categories in liters per year (percentage of TOTAL SUM by year)', showlegend=True)
fig.show()

ТОП-5 категорий занимают более 50% рынка. Существенных изменений в доле рынка по категориям в 2020 году в сравнении с 2019 не произошло. Стоить отметить, что в 2020 выросла до 5% доля рынка по категории Cocktails/RTD.

Посмотрим как изменились покупательские предпочтения в течении трех лет по компаниям производителям (брендам)

In [46]:
brand_percent= df.groupby([df.date.dt.year, 'vendor_name']).agg({'volume_sold_liters': 'sum'})
brand_percent= brand_percent.groupby(level=0).apply(lambda x: 100*x / float(x.sum()))
brand_percent=brand_percent.sort_values(['date','volume_sold_liters'], ascending=False).reset_index()
brand_percent=brand_percent.groupby(['date']).head(5)
fig = go.Figure(data=[
                      go.Bar(name='2020', x=brand_percent[brand_percent['date']==2020].vendor_name, 
                      y=brand_percent[brand_percent['date']==2020].volume_sold_liters, 
                      text=round(brand_percent[brand_percent['date']==2020].volume_sold_liters, 0), textposition='outside'),
                      go.Bar(name='2019', x=brand_percent[brand_percent['date']==2019].vendor_name, 
                      y=brand_percent[brand_percent['date']==2019].volume_sold_liters, 
                      text=round(brand_percent[brand_percent['date']==2019].volume_sold_liters, 0), textposition='outside'), 
                      go.Bar(name='2018', x=brand_percent[brand_percent['date']==2018].vendor_name, 
                      y=brand_percent[brand_percent['date']==2018].volume_sold_liters, 
                      text=round(brand_percent[brand_percent['date']==2018].volume_sold_liters, 0), textposition='outside')            
])   
fig.update_layout(barmode='group', title= 'TOP-5 sales vendor_name in liters per year (percentage of TOTAL SUM by year)', showlegend=True)
fig.show()

В 2020 году 10% рынка занял Heaven Hill Brands (рост +4% относительно 2019), Diageo Americas по сравнению с 2018 потерял 2,2% рынка.

По 2020 году выведем топ 5 позиций в каждом из топ 5 брендов. Так же интересно, какой они имеют объем в литрах

In [47]:
brand_percent_2020=brand_percent[brand_percent.date==2020]
brand_percent_2020.rename(columns={"volume_sold_liters": "%_volume_sold_liters"}, inplace=True)
product_percent= df[df.date.dt.year==2020].groupby([df.date.dt.year, 'vendor_name', 'bottle_volume_liters', 'item_description']).agg({'volume_sold_liters': 'sum'})
product_percent= product_percent.groupby(['date', 'vendor_name']).apply(lambda x: 100*x / float(x.sum()))
product_percent=product_percent.sort_values(['date', 'vendor_name', 'volume_sold_liters'], ascending=False).reset_index()
product_percent=product_percent.groupby(['date', 'vendor_name']).head(5)
product_percent = pd.merge(product_percent, brand_percent_2020, on=['vendor_name'], how="inner")
product_percent.drop(["date_y"], axis=1, inplace=True)
product_percent.rename(columns={'date_x': "date",
                                 "volume_sold_liters": "item_percent_sold_liters_in_total_vendor",
                                 '%_volume_sold_liters': "vendor_percent_sold_liters_in_total_sum"}, 
                      inplace=True)
product_percent=product_percent.round({'item_percent_sold_liters_in_total_vendor': 1, 'vendor_percent_sold_liters_in_total_sum': 1})

fig = go.Figure(data=[go.Table(
    columnwidth = [30,90,60,90,120,120],
    header=dict(values=list(product_percent.columns),
                fill_color='paleturquoise',
                align='center'),
    cells=dict(values=[product_percent.date,
                       product_percent.vendor_name,
                       product_percent.bottle_volume_liters,
                       product_percent.item_description,
                       product_percent.item_percent_sold_liters_in_total_vendor,
                       product_percent.vendor_percent_sold_liters_in_total_sum
                       ],
               fill_color='lavender',
               align='center'))
])

fig.show()


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Только у производителей LUXCO INC и Heaven Hill Brands есть явные продукты-лидеры продаж внутри бренда

Посмотрим как всплеск продаж в марте 2020 года повлиял на прирост продаж разных ценовых сегментов продукции. Для этого для каждого продукта определим среднюю взвешенную цену, затем найдем квартили и медиану цен и постром на их основе ценовые диапазоны.

In [48]:
prod_avg_price= df[df.date.dt.year==2020][['item_description', 
                                          'bottle_volume_liters', 
                                          'bottles_sold', 
                                          'sale_dollars']].groupby(['item_description', 
                                                                    'bottle_volume_liters']).agg({
                                                                                                  'bottles_sold': 'sum',
                                                                                                  'sale_dollars': 'sum'})
prod_avg_price.columns = ['sum_bottles_sold',
                          'sum_sale_dollars'
                          ]

prod_avg_price['avg_price']=prod_avg_price['sum_sale_dollars']/prod_avg_price['sum_bottles_sold']

prod_avg_price.describe()

,sum_bottles_sold,sum_sale_dollars,avg_price
count,4.051000e+03,4.051000e+03,4051.000000
mean,6.060129e+03,7.999083e+04,27.462575
std,3.411288e+04,3.263911e+05,64.829578
min,1.000000e+00,8.310000e+00,1.340000
25%,4.800000e+01,1.079010e+03,10.500000
50%,5.170000e+02,1.083318e+04,17.700000
75%,3.220500e+03,4.356797e+04,30.000000
max,1.734853e+06,8.551140e+06,2623.680000


Таким образом имеем четыре диапазона:

1.   '<=' 11 USD
2.   11-18 USD
3.   18-30 USD
4.   '>'30 USD



Добавим ценовые диапазоны в основной датафрейм

In [49]:
df['price_range']=np.where(
    df['state_bottle_retail'].between(0, 11, inclusive=False), 
    '0-11', 
                  np.where(
    df['state_bottle_retail'].between(11, 18, inclusive=False), 
    '11-18',
                  np.where(
    df['state_bottle_retail'].between(18, 30, inclusive=False), 
    '18-30', '>=30'
     )))

Ранее сравнение продаж месяц к месяцу по годам позволило заметить значительный скачек продаж в марте 2020, что нетипично для данного месяца. По четырем ценовым диапазонам оценим влияние паники связанной с Covid в марте 2020 на прирост в процентом выражении марта к февралю 2020.

In [50]:
feb_mar_2020=pd.pivot_table(df[(df.date.dt.year==2020) & (df.date.dt.month.between(2,3))],index=["price_range"],values="volume_sold_liters",
              columns=df.date.dt.month, aggfunc=np.sum)
feb_mar_2020.reset_index()
feb_mar_2020["mar/feb'20"]=feb_mar_2020[3]/feb_mar_2020[2]-1
feb_mar_2020.rename(columns={2: "feb'20", 3: "mar'20"}, inplace=True)
feb_mar_2020.style.format({"feb'20": "{:.0f}", "mar'20": "{:.0f}", "mar/feb'20": '{:+.1%}'})

date,feb'20,mar'20,mar/feb'20
price_range,,,
0-11,547965,633537,+15.6%
11-18,589325,704719,+19.6%
18-30,402907,491274,+21.9%
>=30,90278,126537,+40.2%


Наиболее значимый прирост (+40%) показал сегмент от 30 долларов и выше в то время как остальные сегменты приросли от 15% до 22%. Эти данные необходимо учесть при планировании запасов, так как очевидно, что объемы страховых запасов более дорогой продукции значительно ниже чем дешевой. И в случае прогнозов подобного поведения покупателей необходимо принять меры по обеспечении торговых точек продукцией.